#### Pandas for Dataprocessing Sample

In [1]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

In [2]:
stock = pd.read_csv("D:\databank\CSMAR\TRD_Mnth.csv")
stock = stock[['Stkcd','Trdmnt','Mclsprc','Msmvosd','Msmvttl','Ndaytrd','Mretwd','Mretnd','Markettype']].copy()

#### Filters 
stock = stock[stock['Markettype'].isin([1,4,16,32])].copy() # A share stocks

stock = stock[(stock['Trdmnt']>199800) & (stock['Trdmnt']<=202012)].copy()   # Sample Period 2000 - 2020

In [3]:
stock['ym'] = stock['Trdmnt'].apply(lambda x:(x//100)*12 + x%100)

In [4]:
### 一种补全缺失行的方法

ct = pd.DataFrame()
stkcd = []
ym = []
for x in stock.groupby(['Stkcd'])['ym'].agg(['first','last']).itertuples():
    ym += list(range(x[1],x[2]+1))
    stkcd += [x[0]] * (x[2]-x[1]+1)
    
ct['Stkcd'] = stkcd
ct['ym'] = ym
stock = pd.merge(stock,ct,on=['Stkcd','ym'],how='outer')
stock.sort_values(by=['Stkcd','ym'],inplace=True)

In [13]:
cap = 'Msmvttl'
date = 'ym'
#stock['cap'] = stock.groupby(['Stkcd'])[cap].shift(1)
stock['cap'] = stock[cap]
tp = stock.groupby([date])['cap'].quantile(0.1).reset_index() # drop 10% smallest stock 
tp.columns = [date,'smallest10p']

stock = pd.merge(stock,tp,on=[date])
stock.loc[stock['cap'] <= stock['smallest10p'],'Dretwd'] = np.nan
stock.drop(columns=['smallest10p'],inplace=True)

stock.loc[(stock['Mclsprc']<3),'Dretwd'] = np.nan  # drop stock which has price less than 3 ￥


stock.sort_values(by=['Stkcd',date],inplace=True)
#stock.dropna(subset=['Mretwd'],inplace=True)

#### All Sample

In [6]:
### MOM12

stock['Mretwd_lg'] = np.log(stock['Mretwd'] + 1)
stock['cumret12'] = (np.exp(stock.groupby(['Stkcd'])['Mretwd_lg'].rolling(11).sum()) - 1).reset_index(level=0)['Mretwd_lg']
stock['cumret12_lag'] = stock.groupby(['Stkcd'])['cumret12'].shift(1)

stock['cumret_quintile'] = stock.dropna(subset=['cumret12_lag']).groupby([date])['cumret12_lag'].transform(lambda x:pd.qcut(x,5,labels=False))

data = stock[(stock[date]>2000*12)].copy()
tp = data.groupby([date,'cumret_quintile'])['Mretwd'].mean().reset_index().pivot(index=date,columns='cumret_quintile',values='Mretwd')
tp = (tp[4] - tp[0])

print(tp.mean()*100, tp.mean() / tp.std() * tp.count() ** 0.5)

-0.5118742329517177 -1.5058654796374722


In [7]:
### short-term reversal

stock['ret_lag'] = stock.groupby(['Stkcd'])['Mretwd'].shift(1)

stock['lastret_quintile'] = stock.dropna(subset=['ret_lag']).groupby([date])['ret_lag'].transform(lambda x:pd.qcut(x,5,labels=False))

data = stock[(stock[date]>=2000*12)].copy()
tp = data.groupby([date,'lastret_quintile'])['Mretwd'].mean().reset_index().pivot(index=date,columns='lastret_quintile',values='Mretwd')
tp = (tp[4] - tp[0])

print(tp.mean()*100, tp.mean() / tp.std() * tp.count() ** 0.5)

-1.1841864734539431 -4.179920826544981
